In [ ]:
import os
import glob
import re
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

RAW_DIR = "./dataset_global/raw_hourly"

class WeatherDataset(Dataset):
    """
    从 parquet 读取天气样本，按雨量分段做重采样；支持下采样与跳样以控制体积。
    训练脚本中会对 features 做标准化，对目标做 log1p 变换。
    自动检测并修复缺失 time 列的文件。
    """
    def __init__(
        self,
        root=RAW_DIR,
        sample_frac=0.2,
        skip_rate=3,
        bin_weights=None,
        random_state=42
    ):
        self.files = sorted(glob.glob(os.path.join(root, "*.parquet")))
        if not self.files:
            raise RuntimeError(f"未在 {root} 找到 parquet 文件。")

        self.sample_frac = sample_frac
        self.skip_rate = skip_rate
        self.random_state = random_state
        self.time_ok = 0
        self.time_missing = 0

        if bin_weights is None:
            bin_weights = {"none": 1, "light": 10, "moderate": 20, "heavy": 100, "extreme": 300}
        self.bin_weights = bin_weights

        parts = []
        for f in self.files:
            try:
                df = pd.read_parquet(f)
                if df.empty:
                    continue

                # 尝试恢复时间列（可能是 index）
                if "time" not in df.columns and isinstance(df.index, pd.DatetimeIndex):
                    df = df.reset_index().rename(columns={"index": "time"})

                # 若仍然没有 time，则根据文件名推断
                if "time" not in df.columns:
                    m = re.search(r"_(\d{4}-\d{2}-\d{2})_(\d{4}-\d{2}-\d{2})\.parquet$", f)
                    if m:
                        start = pd.to_datetime(m.group(1))
                        end = pd.to_datetime(m.group(2))
                        n = len(df)
                        df["time"] = pd.date_range(start, periods=n, freq="H")
                    else:
                        df["time"] = pd.NaT

                if df["time"].isna().all():
                    self.time_missing += 1
                else:
                    self.time_ok += 1

                # 下采样（每个文件）
                df = df.sample(frac=self.sample_frac, random_state=self.random_state)

                # 时间字段展开
                t = pd.to_datetime(df["time"], errors="coerce")
                df["hour"] = t.dt.hour.fillna(0).astype(int)
                df["month"] = t.dt.month.fillna(1).astype(int)

               
                df["hour_sin"] = np.sin(2*np.pi*df["hour"]/24)
                df["hour_cos"] = np.cos(2*np.pi*df["hour"]/24)
                df["month_sin"] = np.sin(2*np.pi*df["month"]/12)
                df["month_cos"] = np.cos(2*np.pi*df["month"]/12)

                # 雨量分箱
                bins = [-0.1, 0.1, 1.0, 5.0, 20.0, np.inf]
                labels = ["none", "light", "moderate", "heavy", "extreme"]
                df["rain_bin"] = pd.cut(df["precipitation"], bins=bins, labels=labels)

                # 重采样
                resampled = []
                for level in labels:
                    grp = df[df["rain_bin"] == level]
                    if len(grp) == 0:
                        continue
                    w = self.bin_weights.get(level, 1)
                    n = max(1, int(len(grp) * w))
                    resampled.append(grp.sample(n=n, replace=True, random_state=self.random_state))
                if not resampled:
                    continue

                merged = pd.concat(resampled, ignore_index=True)
                merged = merged.iloc[::self.skip_rate, :]
                parts.append(merged)

            except Exception as e:
                print(f"⚠️ 跳过 {f}: {e}")

        if not parts:
            raise RuntimeError("没有构建出有效样本。")

        self.data = pd.concat(parts, ignore_index=True).reset_index(drop=True)
        drop_cols = {"precipitation", "rain_bin", "time"}
        self.features = [c for c in self.data.columns if c not in drop_cols]
        self.mean = self.data[self.features].mean()
        self.std = self.data[self.features].std().replace(0, 1e-6)
        self.data[self.features] = (self.data[self.features] - self.mean) / (self.std + 1e-6)
        print("✅ 已完成特征标准化（均值≈0，方差≈1）")


        print(f"✅ Loaded {len(self.data)} samples from {len(self.files)} files.")
        print(f"🕒 time列存在于 {self.time_ok} 个文件，缺失 {self.time_missing} 个文件。")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # ✅ 安全转换为 float32，忽略非法值
        x = torch.tensor(row[self.features].astype(float).values, dtype=torch.float32)
        y = torch.tensor(float(row["precipitation"]), dtype=torch.float32)
        return x, y



# ---------------- main 预览部分 ---------------- #
if __name__ == "__main__":
    ds = WeatherDataset(sample_frac=0.1, skip_rate=5)
    df = ds.data

    print("\n📊 数据集总体信息:")
    print(df.describe(include='all').transpose().head(10))

    print("\n🌧️ 各雨量区间样本数量:")
    print(df["rain_bin"].value_counts().sort_index())

    print("\n🌡️ 特征列示例:")
    print(ds.features[:10])

    print("\n🔍 随机查看部分样本:")
    print(df.sample(5, random_state=0)[["precipitation", "rain_bin"] + ds.features[:5]])
    print(ds.data.dtypes[ds.data.dtypes == 'object'])


✅ 已完成特征标准化（均值≈0，方差≈1）
✅ Loaded 140523 samples from 3840 files.
🕒 time列存在于 3840 个文件，缺失 0 个文件。

📊 数据集总体信息:
                         count unique  top freq  \
time                    140523    NaN  NaN  NaN   
temperature_2m        140523.0    NaN  NaN  NaN   
relative_humidity_2m  140523.0    NaN  NaN  NaN   
surface_pressure      140523.0    NaN  NaN  NaN   
wind_speed_10m        140523.0    NaN  NaN  NaN   
wind_direction_10m    140523.0    NaN  NaN  NaN   
cloud_cover           140523.0    NaN  NaN  NaN   
precipitation         140523.0    NaN  NaN  NaN   
hour                  140523.0    NaN  NaN  NaN   
month                 140523.0    NaN  NaN  NaN   

                                               mean                  min  \
time                  2020-01-18 20:16:13.403642368  2015-01-01 10:00:00   
temperature_2m                                  0.0            -6.317824   
relative_humidity_2m                           -0.0            -4.359904   
surface_pressure             

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, random_split, Dataset
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
import pandas as pd


# =========================================================
# ⚙️ 数据包装器：将 WeatherDataset 转为时间序列样本
# =========================================================
class SequenceWrapper(Dataset):
    """将 WeatherDataset 转换为 [seq_len → 预测下一步] 样本"""
    def __init__(self, base_ds: WeatherDataset, seq_len=24):
        self.base = base_ds
        self.seq_len = seq_len
        self.df = base_ds.data.sort_values("time").reset_index(drop=True)
        self.features = base_ds.features
        self.values = self.df[self.features].values.astype(np.float32)
        self.targets = self.df["precipitation"].values.astype(np.float32)

    def __len__(self):
        return len(self.df) - self.seq_len

    def __getitem__(self, idx):
        x = self.values[idx:idx + self.seq_len]
        y = self.targets[idx + self.seq_len]
        return torch.tensor(x), torch.tensor(y)


# =========================================================
# 🧠 Transformer 模型
# =========================================================
class RainfallTransformer(nn.Module):
    def __init__(self, input_dim, seq_len=24, d_model=256, nhead=8, num_layers=4, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_embedding = nn.Parameter(torch.randn(1, seq_len, d_model))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=4 * d_model,
            dropout=dropout,
            activation="gelu",
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.head = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, 256),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(256, 1),
        )

    def forward(self, x):
        # x: [B, seq_len, input_dim]
        x_proj = self.input_proj(x) + self.pos_embedding[:, :x.size(1)]
        enc = self.encoder(x_proj)                # [B, seq_len, D]
        pooled = enc.mean(dim=1)                  # 平均池化
        out = self.head(pooled).squeeze(-1)
        return out


# =========================================================
# 训练相关工具
# =========================================================
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    print(f"🧠 使用设备: {device}")
    return device


class EarlyStopping:
    def __init__(self, patience=6, min_delta=1e-4):
        self.best = float("inf")
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.stop = False

    def step(self, v):
        if v < self.best - self.min_delta:
            self.best = v
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.stop = True


def plot_loss(history, out="loss_curve.png"):
    plt.figure(figsize=(8, 5))
    plt.plot(history["train"], label="Train Loss", marker="o")
    plt.plot(history["val"], label="Val Loss", marker="s")
    plt.xlabel("Epoch"); plt.ylabel("Huber Loss")
    plt.title("Training & Validation Loss")
    plt.legend(); plt.grid(True); plt.tight_layout()
    plt.savefig(out, dpi=150); plt.close()
    print(f"📊 训练曲线已保存为 {out}")


def scatter_with_error(trues, preds, out="pred_vs_true_colored.png"):
    plt.figure(figsize=(6, 6))
    errors = preds - trues
    sc = plt.scatter(trues, preds, c=errors, cmap="coolwarm", s=8, alpha=0.7)
    lim = max(trues.max(), preds.max())
    plt.plot([0, lim], [0, lim], "k--", lw=1)
    plt.colorbar(sc, label="Pred - True (log1p)")
    plt.xlabel("True Precipitation (log1p)")
    plt.ylabel("Predicted (log1p)")
    plt.title("Prediction vs Actual (Color = Error)")
    plt.grid(True); plt.tight_layout()
    plt.savefig(out, dpi=150); plt.close()
    print(f"📈 预测散点图已保存为 {out}")


def compute_metrics(trues, preds):
    rmse = np.sqrt(mean_squared_error(trues, preds))
    mae = mean_absolute_error(trues, preds)
    r2 = r2_score(trues, preds)
    return rmse, mae, r2


# =========================================================
# 🏋️‍♂️ 训练主循环
# =========================================================
def train_model(model, train_loader, val_loader, device, epochs=50, base_lr=5e-4, use_ema=True):
    model = model.to(device)
    criterion = nn.HuberLoss(delta=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=base_lr, weight_decay=1e-3)
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=base_lr * 5.0, epochs=epochs, steps_per_epoch=len(train_loader)
    )

    ema_model = deepcopy(model) if use_ema else None
    ema_decay = 0.999
    best_val = float("inf")
    history = {"train": [], "val": []}
    early_stop = EarlyStopping(patience=6, min_delta=1e-4)

    for epoch in range(1, epochs + 1):
        # ---- train ----
        model.train()
        total = 0.0
        for xb, yb in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs} [train]", unit="batch"):
            xb, yb = xb.to(device), yb.to(device)
            yb_log = torch.log1p(yb)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb_log)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            if use_ema:
                with torch.no_grad():
                    for p, ep in zip(model.parameters(), ema_model.parameters()):
                        ep.data.mul_(ema_decay).add_(p.data, alpha=1 - ema_decay)
            total += loss.item() * len(xb)

        train_loss = total / len(train_loader.dataset)
        history["train"].append(train_loss)

        # ---- val ----
        model.eval()
        val_total = 0.0
        preds_all, trues_all = [], []
        with torch.no_grad():
            eval_model = ema_model if (use_ema and epoch > 1) else model
            for xb, yb in tqdm(val_loader, desc=f"Epoch {epoch}/{epochs} [val]", unit="batch"):
                xb, yb = xb.to(device), yb.to(device)
                yb_log = torch.log1p(yb)
                pred = eval_model(xb)
                val_total += criterion(pred, yb_log).item() * len(xb)
                preds_all.append(pred.detach().cpu())
                trues_all.append(yb_log.detach().cpu())

        val_loss = val_total / len(val_loader.dataset)
        history["val"].append(val_loss)

        preds_np = torch.cat(preds_all).numpy()
        trues_np = torch.cat(trues_all).numpy()
        rmse, mae, r2 = compute_metrics(trues_np, preds_np)
        print(f"📉 Epoch {epoch:>2} | Train {train_loss:.6f} | Val {val_loss:.6f} | RMSE {rmse:.4f} | MAE {mae:.4f} | R² {r2:.4f}")

        if val_loss < best_val:
            best_val = val_loss
            torch.save((ema_model if (use_ema and epoch > 1) else model).state_dict(), "checkpoint_best.pt")
            print(f"💾 新最佳模型已保存 (val_loss={val_loss:.6f})")

        early_stop.step(val_loss)
        if early_stop.stop:
            print(f"⏹️ Early stopping at epoch {epoch}")
            break

    plot_loss(history)
    return history, (ema_model if use_ema else model)


# =========================================================
# 📊 验证集可视化
# =========================================================
def visualize_on_val(model, val_loader, device):
    import numpy as np
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            pred = model(xb)
            preds.append(pred.cpu())
            trues.append(torch.log1p(yb).cpu())  # log 空间评估
    preds = torch.cat(preds).numpy()
    trues = torch.cat(trues).numpy()

    # ---------- log 空间 ----------
    rmse, mae, r2 = compute_metrics(trues, preds)
    print(f"🔎 Final (Val, log space) → RMSE={rmse:.4f}, MAE={mae:.4f}, R²={r2:.4f}")
    scatter_with_error(trues, preds, out="pred_vs_true_colored.png")

    # ---------- 转回 mm/h 空间 ----------
    preds_lin = np.expm1(preds)
    trues_lin = np.expm1(trues)
    rmse_lin, mae_lin, r2_lin = compute_metrics(trues_lin, preds_lin)
    print(f"🌧️  (Val, mm/h) → RMSE={rmse_lin:.4f}, MAE={mae_lin:.4f}, R²={r2_lin:.4f}")

    # ---------- 绘制线性空间散点 ----------
    plt.figure(figsize=(6,6))
    errors = preds_lin - trues_lin
    sc = plt.scatter(trues_lin, preds_lin, c=errors, cmap="coolwarm", s=8, alpha=0.7)
    lim = max(trues_lin.max(), preds_lin.max()) * 1.05
    plt.plot([0, lim], [0, lim], "k--", lw=1)
    plt.xscale("log"); plt.yscale("log")
    plt.colorbar(sc, label="Pred - True (mm/h)")
    plt.xlabel("True Precipitation (mm/h)")
    plt.ylabel("Predicted (mm/h)")
    plt.title("Prediction vs Actual (Color = Error, log-log scale)")
    plt.grid(True, which="both", ls="--", lw=0.5)
    plt.tight_layout()
    plt.savefig("pred_vs_true_real_mm.png", dpi=150)
    plt.close()
    print("📈 已保存真实单位空间散点图：pred_vs_true_real_mm.png")



# =========================================================
# main
# =========================================================
if __name__ == "__main__":
    device = get_device()

    base_ds = WeatherDataset(sample_frac=0.15, skip_rate=3)
    feature_cols = base_ds.features

    # ⚙️ 标准化
    base_ds.mean = base_ds.data[feature_cols].mean()
    base_ds.std = base_ds.data[feature_cols].std().replace(0, 1)
    base_ds.data[feature_cols] = (base_ds.data[feature_cols] - base_ds.mean) / (base_ds.std + 1e-6)

    # 构建时间序列样本
    seq_len = 24
    ds_seq = SequenceWrapper(base_ds, seq_len=seq_len)

    # 划分训练/验证
    val_size = int(0.2 * len(ds_seq))
    train_size = len(ds_seq) - val_size
    train_ds, val_ds = random_split(ds_seq, [train_size, val_size], generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_ds, batch_size=256, shuffle=True, num_workers=0, pin_memory=False)
    val_loader = DataLoader(val_ds, batch_size=256, shuffle=False, num_workers=2, pin_memory=False)

    # 初始化模型
    model = RainfallTransformer(input_dim=len(feature_cols), seq_len=seq_len)
    history, final_model = train_model(model, train_loader, val_loader, device, epochs=10, base_lr=1e-5, use_ema=True)

    # 加载最优模型
    best = RainfallTransformer(input_dim=len(feature_cols), seq_len=seq_len).to(device)
    best.load_state_dict(torch.load("checkpoint_best.pt", map_location=device))
    best.eval()

    visualize_on_val(best, val_loader, device)
    print("✅ 训练与评估完成。最优权重: checkpoint_best.pt")


🧠 使用设备: mps
✅ 已完成特征标准化（均值≈0，方差≈1）
✅ Loaded 352367 samples from 3840 files.
🕒 time列存在于 3840 个文件，缺失 0 个文件。


Epoch 1/10 [val]:   0%|          | 0/276 [00:00<?, ?batch/s]Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/yangliu/miniforge3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yangliu/miniforge3/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'SequenceWrapper' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
Epoch 1/10 [val]:   0%|          | 0/276 [02:04<?, ?batch/s]


KeyboardInterrupt: 